# Speed-ups with single-precision floating points

Using single precision floating points can improve the speed of training the models by atleast a factor of 2. The tradeoff is a small loss of accuracy and non-deterministic behavior due to rounding errors.

In [1]:
import torch
import math
import time
import random
import numpy as np

from lvgp_pytorch.models import LVGPR
from lvgp_pytorch.optim import fit_model_scipy,noise_tune
from lvgp_pytorch.utils.variables import NumericalVariable,CategoricalVariable
from lvgp_pytorch.utils.input_space import InputSpace

from typing import Dict
from copy import deepcopy

def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)

## Generating the training and test sets

We will be using the piston function for demonstration. The piston function is given by

$$
2\pi\sqrt{\frac{M}{
    k+ S^2\frac{P_0V_0}{V^2}\frac{T}{T_o}}},
$$

where $V = \frac{S}{2k}\sqrt{A^2+4k\frac{P_0}{T_0}T}$,  $A = P_0S + 19.62 M - \frac{kV_0}{S}$, and the 7 inputs are ($M,S,V_0,k,P_o,$ $T,T_0$). All 7 inputs are numerical. Similar to [Zhang et al. (2020)](https://doi.org/10.1080/00401706.2019.1638834), we discretize $P_0$ and $k$ over their domains to have 5 and 9 levels respectively.

In [2]:
# reponse function
def piston(params:Dict)->float:
    A = params['P_0']*params['S'] + 19.62*params['M'] - params['k']*params['V_0']/params['S']
    term1 = params['P_0']*params['V_0']/params['T_0']*params['T_a']
    V = params['S']/2/params['k']*(math.sqrt(A**2 + 4*term1)-A)
    term2 = params['k'] + (params['S']**2)*term1/(V**2)
    return 2*math.pi*math.sqrt(params['M']/term2)

# InputSpace object
config = InputSpace()
M = NumericalVariable(name='M',lower=30,upper=60)
S = NumericalVariable(name='S',lower=0.005,upper=0.02)
V0 = NumericalVariable(name='V_0',lower=0.002,upper=0.01)
Ta = NumericalVariable(name='T_a',lower=290,upper=296)
T0 = NumericalVariable(name='T_0',lower=340,upper=360)

P0 = CategoricalVariable(name='P_0',levels=np.linspace(90000,110000,5))
k = CategoricalVariable(name='k',levels=np.linspace(1000,5000,9))
config.add_inputs([M,S,V0,Ta,T0,P0,k])
config

Input space with variables:

M, Type: Numerical, Range: [30.0,60.0]
S, Type: Numerical, Range: [0.005,0.02]
V_0, Type: Numerical, Range: [0.002,0.01]
T_a, Type: Numerical, Range: [290.0,296.0]
T_0, Type: Numerical, Range: [340.0,360.0]
P_0, Type: Categorical, Levels: {90000.0, 95000.0, 100000.0, 105000.0, 110000.0}
k, Type: Categorical, Levels: {1000.0, 1500.0, 2000.0, 2500.0, 3000.0, 3500.0, 4000.0, 4500.0, 5000.0}

We will generate 500 random samples to be used as training data and separately 1000 random samples to be used as test data. Note that data to be supplied to the model for training and predictions are in double precision.

In [3]:
# generate 500 samples
set_seed(1)
num_samples = 500
train_x = torch.from_numpy(
    config.random_sample(np.random,num_samples)
)
train_y = [None]*num_samples

for i,x in enumerate(train_x):
    train_y[i] = piston(config.get_dict_from_array(x.numpy()))

train_y = torch.tensor(train_y).double()


# generate 1000 test samples
num_samples = 1000
test_x = torch.from_numpy(config.random_sample(np.random,num_samples))
test_y = [None]*num_samples

for i,x in enumerate(test_x):
    test_y[i] = piston(config.get_dict_from_array(x.numpy()))
    
# create tensor objects
test_y = torch.tensor(test_y).to(train_y)

## Model with double-precision


To create a model instance with double precision, use the `.double()` method. Test data for which predictions are to be computed need to be in double precision format as well. 

In [4]:
# create LVGP instance
set_seed(4)
model = LVGPR(
    train_x=train_x,
    train_y=train_y,
    qual_index=config.qual_index,
    quant_index=config.quant_index,
    num_levels_per_var=list(config.num_levels.values()),
    quant_correlation_class="RBFKernel",
).double()

# fit model with 10 different starts
start_time = time.time()
_,nll_inc = fit_model_scipy(
    model,
    num_restarts=9, # number of starting points
)
fit_time = time.time()-start_time

print('NLL for double-precision model...: %6.3f'%nll_inc)
print('Training time (seconds)..........: %6.2f'%fit_time)

with torch.no_grad():
    # set return_std = False if standard deviation is not needed
    # test_x needs to be double precision 
    test_mean = model.predict(test_x,return_std=False)
    
# print RRMSE
rrmse = torch.sqrt(((test_y-test_mean)**2).mean()/((test_y-test_y.mean())**2).mean())
print('Test RRMSE.......................: %6.3f'%rrmse.item())

NLL for double-precision model...: -113.811
Training time (seconds)..........: 208.79
Test RRMSE.......................:  0.202


## Model with single-precision


To create a model instance with double precision, use the `.single()` method. Test data for which predictions are to be computed need to be in single precision format as well. For a fair comparison, we use the same starting points for multi-start optimization.

In [5]:
# create LVGP instance
set_seed(4)
model_float = LVGPR(
    train_x=train_x,
    train_y=train_y,
    qual_index=config.qual_index,
    quant_index=config.quant_index,
    num_levels_per_var=list(config.num_levels.values()),
    quant_correlation_class="RBFKernel",
).float() # changing this to float to use single-precision floating point

# fit model with 10 different starts
start_time = time.time()
_,nll_inc_float = fit_model_scipy(
    model_float,
    num_restarts=9, # number of starting points
)
fit_time_float = time.time()-start_time

print('NLL for single-precision model...: %6.3f'%nll_inc_float)
print('Training time (seconds)..........: %6.2f'%fit_time_float)

with torch.no_grad():
    # set return_std = False if standard deviation is not needed
    # since test_x is in double precisions, supply a single precision
    # version test_x.float()
    test_mean = model_float.predict(test_x.float(),return_std=False)
    
# print RRMSE
rrmse_float = torch.sqrt(
    ((test_y-test_mean.double())**2).mean()/((test_y-test_y.mean())**2).mean()
)
print('Test RRMSE.......................: %6.3f'%rrmse_float.item())

NLL for single-precision model...: -113.534
Training time (seconds)..........:  54.35
Test RRMSE.......................:  0.203


In this case, the model single precision model takes about a quarter of the training time of the double precision model to train with a minor loss in accuracy. Note that due to the non-deterministic behavior due to rounding errors, the results obtained may slightly differ in different runs.